<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 22.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import pytorch_lightning as pl
#from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [3]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [6]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [7]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [8]:
train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")
validation = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/validation.csv")
test = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/boneage-test-dataset.csv")

In [9]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [10]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['boneage'] = train['boneage'].apply(lambda x: str(x))
train['male'] = train['male'].apply(lambda x: 0 if x else 1)

df1 = pd.DataFrame(train)
df1.rename(columns={'male':'gender'},inplace=True)

In [11]:
train

,id,boneage,gender
0,1377.png,180,1
1,1378.png,12,1
2,1379.png,94,1
3,1380.png,120,0
4,1381.png,82,1
...,...,...,...
10084,13259.png,94,1
10085,13261.png,94,1
10086,13274.png,94,1
10087,13286.png,94,1


In [12]:
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
validation['boneage'] = validation['boneage'].apply(lambda x: str(x))
validation['male'] = validation['male'].apply(lambda x: 0 if x else 1)

df2 = pd.DataFrame(validation)
df2.rename(columns={'male':'gender'},inplace=True)

In [13]:
validation

,id,boneage,gender
0,10186.png,10,0
1,10758.png,15,1
2,10860.png,180,1
3,10877.png,180,1
4,11008.png,180,1
...,...,...,...
2517,15605.png,50,1
2518,15606.png,113,1
2519,15608.png,55,1
2520,15609.png,150,0


In [14]:
test

,Case ID,Sex,Ground truth bone age (months)
0,4360,M,169
1,4361,M,170
2,4362,M,73
3,4363,M,153
4,4364,M,135
...,...,...,...
195,4555,F,133
196,4556,F,129
197,4557,F,168
198,4558,F,136


In [15]:
test['Case ID'] = test['Case ID'].apply(lambda x: str(x)+'.png')
#test['boneage'] = test['boneage'].apply(lambda x: str(x))
#test['Sex'] = test['Sex'].apply(lambda x: 0 if x else 1)

df3 = pd.DataFrame(test)
df3.rename(columns={'Case ID':'id'},inplace=True)
df3.rename(columns={'Sex':'gender'},inplace=True)
df3.rename(columns={'Ground truth bone age (months)':'boneage'},inplace=True)
df3['gender'].replace({ 'M':0,'F':1}, inplace=True)

test= test[['id','boneage','gender']]

In [16]:
test

,id,boneage,gender
0,4360.png,169,0
1,4361.png,170,0
2,4362.png,73,0
3,4363.png,153,0
4,4364.png,135,0
...,...,...,...
195,4555.png,133,1
196,4556.png,129,1
197,4557.png,168,1
198,4558.png,136,1


#clone

##ลองดึง Inception_V3 จาก github ตัวเอง

In [17]:
#Local imports
#from Project_Boneage.backbone.inception_v3 import InceptionV3

##ลองดึง Inception_V3 จาก github ของ Bonet มาใช้ก่อน

In [18]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet


Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [19]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

#Creating a custom PyTorch Dataset

In [20]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, pickle_file, image_dir):
        self.image_dir = image_dir
        self.pickle_file = pickle_file
        self.tabular = pd.read_pickle(pickle_file)

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        y = tabular["boneage"]

        image = Image.open(f"{self.image_dir}/{tabular['id']}.png")
        image = np.array(image)
        image = image[..., :3]

        image = transforms.functional.to_tensor(image)

        tabular = tabular[["gender"]]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return image, tabular, y

In [21]:
import torchvision.models as models
inception = models.inception_v3(init_weights=True)

In [22]:
#def conv_block(input_size, output_size):
#    block = nn.Sequential(
#        nn.Conv2d(input_size, output_size, (3, 3)), nn.ReLU(), nn.BatchNorm2d(output_size), nn.MaxPool2d((2, 2)),
#    )
#
#    return block

In [63]:
class BoNetModule(pl.LightningModule):

    def __init__(
        self, transform_input = False, lr = 0.0001, num_workers = 0, batch_size = 20
    ):

        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = inception#InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x, y):
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

    def training_step(self, batch, batch_idx):
      image, tabular, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, tabular))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)

      tensorboard_logs = {"train_loss": loss}
      return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

    def setup(self, stage):

        image_datatrain = ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")
        image_dataval = ImageDataset(pickle_file=path+"/validation.csv", image_dir=f"{val_img}")
        image_datatest = ImageDataset(pickle_file=path+"/boneage-test-dataset.csv", image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest

        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr))

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    

class DenseLayer(pl.LightningModule):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)



#Training

In [24]:
from fastai.vision.learner import Learner
from fastai.callback.schedule import lr_find

In [25]:
logger = TensorBoardLogger("lightning_logs", name="image_only")
early_stop_callback = EarlyStopping(monitor="val_loss",min_delta=5000,patience=7, verbose=False, mode="min")
    

In [40]:
model = BoNetModule()
#trainer = pl.Trainer(accelerator='cpu', logger=logger, callbacks=early_stop_callback)


In [27]:
START_EPOCH=0
NUM_EPOCHS=60
LR=0.0001
PATIENCE=2
BATCH_SIZE=20
NUM_WORKERS=0
NUM_GPUS=1
GPUS=0

In [28]:
# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=LR, weight_decay = 0.005, momentum = 0.9)  

#total_step = len(image_datatrain)

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [56]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(NUM_EPOCHS):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(self):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1,NUM_EPOCHS, loss.item()))

            
            


NameError: ignored

In [35]:
F.relu

<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>

In [ ]:
lr_finder = trainer.lr_find(model)
fig = lr_finder.plot(suggest=True, show=True)
new_lr = lr_finder.suggestion()
print(new_lr)
model.hparams.lr = new_lr

trainer.fit(model)
trainer.test(model)

In [ ]:
trainer.fit(model)
trainer.test(model)

In [ ]:
tensorboard --logdir=lightning_log